# 🔍 Intermodal Transportation - Data Exploration

**Objective:** Understand the structure, quality, and characteristics of our transportation data.

**Key Questions:**
1. What is the data quality status?
2. What are the main transport modes?
3. What are the cost and time patterns?
4. Are there any data quality issues?

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries loaded successfully")

## 1️⃣ Load Processed Data

In [ ]:
# Load processed data
df = pd.read_parquet('../data/processed/shipments_processed.parquet')

print(f"📊 Dataset loaded: {len(df):,} records")
print(f"📅 Date range: {df['shipment_date'].min()} to {df['shipment_date'].max()}")
print(f"📏 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

df.head(10)

## 2️⃣ Data Structure & Quality

In [ ]:
# Data structure
print("📋 DATA STRUCTURE")
print("=" * 60)
df.info()

print("\n📊 STATISTICAL SUMMARY")
print("=" * 60)
df.describe().T

In [ ]:
# Missing values analysis
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
}).sort_values('Missing Count', ascending=False)

missing_df = missing_df[missing_df['Missing Count'] > 0]

if len(missing_df) > 0:
    print("⚠️ MISSING VALUES DETECTED:")
    print(missing_df)
else:
    print("✅ NO MISSING VALUES - Excellent data quality!")

## 3️⃣ Transport Mode Analysis

In [ ]:
# Transport mode distribution
mode_counts = df['transport_mode'].value_counts()

fig = px.pie(
    values=mode_counts.values,
    names=mode_counts.index,
    title='📦 Shipments by Transport Mode',
    hole=0.4,
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=500, showlegend=True)
fig.show()

print("\n📊 Transport Mode Summary:")
print(mode_counts)

## 4️⃣ Cost Analysis

In [ ]:
# Cost distribution by transport mode
fig = px.box(
    df,
    x='transport_mode',
    y='cost',
    color='transport_mode',
    title='💰 Cost Distribution by Transport Mode',
    labels={'cost': 'Cost (USD)', 'transport_mode': 'Transport Mode'},
    color_discrete_sequence=px.colors.qualitative.Bold
)

fig.update_layout(height=500, showlegend=False)
fig.show()

# Summary statistics
cost_summary = df.groupby('transport_mode')['cost'].agg(['mean', 'median', 'min', 'max']).round(2)
print("\n💵 Cost Statistics by Mode:")
print(cost_summary)

In [ ]:
# Cost efficiency (cost per kg)
if 'cost_per_kg' in df.columns:
    fig = px.violin(
        df,
        x='transport_mode',
        y='cost_per_kg',
        color='transport_mode',
        title='📊 Cost Efficiency (Cost per KG) by Transport Mode',
        box=True,
        points='outliers'
    )
    
    fig.update_layout(height=500, showlegend=False)
    fig.show()

## 5️⃣ Transit Time Analysis

In [ ]:
# Transit time by mode
if 'transit_days' in df.columns:
    fig = px.histogram(
        df,
        x='transit_days',
        color='transport_mode',
        title='⏱️ Transit Time Distribution by Transport Mode',
        nbins=50,
        marginal='box',
        labels={'transit_days': 'Transit Days'}
    )
    
    fig.update_layout(height=500, barmode='overlay')
    fig.update_traces(opacity=0.7)
    fig.show()
    
    # Summary
    transit_summary = df.groupby('transport_mode')['transit_days'].agg(['mean', 'median', 'min', 'max']).round(1)
    print("\n⏰ Transit Time Statistics:")
    print(transit_summary)

## 6️⃣ Cost vs Time Trade-off

In [ ]:
# Scatter: Cost vs Transit Time
if 'transit_days' in df.columns:
    fig = px.scatter(
        df,
        x='transit_days',
        y='cost',
        color='transport_mode',
        size='weight' if 'weight' in df.columns else None,
        title='💰⏱️ Cost vs Transit Time Trade-off',
        labels={'transit_days': 'Transit Days', 'cost': 'Cost (USD)'},
        hover_data=['origin', 'destination'] if 'origin' in df.columns else None,
        opacity=0.6
    )
    
    fig.update_layout(height=600)
    fig.show()

## 7️⃣ Route Analysis

In [ ]:
# Top routes
if 'origin' in df.columns and 'destination' in df.columns:
    df['route'] = df['origin'] + ' → ' + df['destination']
    top_routes = df['route'].value_counts().head(10)
    
    fig = px.bar(
        x=top_routes.values,
        y=top_routes.index,
        orientation='h',
        title='🚢 Top 10 Routes by Volume',
        labels={'x': 'Number of Shipments', 'y': 'Route'},
        color=top_routes.values,
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(height=500, showlegend=False)
    fig.show()

## 8️⃣ Temporal Trends

In [ ]:
# Shipments over time
if 'shipment_date' in df.columns:
    df_time = df.set_index('shipment_date').resample('M').size().reset_index(name='shipments')
    
    fig = px.line(
        df_time,
        x='shipment_date',
        y='shipments',
        title='📈 Shipment Volume Over Time',
        labels={'shipment_date': 'Month', 'shipments': 'Number of Shipments'},
        markers=True
    )
    
    fig.update_layout(height=500)
    fig.show()

## 9️⃣ Key Insights Summary

### 📊 Data Quality
- **Records:** {record_count}
- **Completeness:** {completeness}%
- **Quality Score:** {quality_score}/10

### 🚢 Transport Modes
- **Most Used:** {most_used_mode}
- **Most Expensive:** {most_expensive_mode}
- **Fastest:** {fastest_mode}

### 💡 Business Insights
1. **Cost Optimization:** {insight_1}
2. **Route Efficiency:** {insight_2}
3. **Recommendations:** {insight_3}

---

**Next Steps:**
- Deep dive into delay analysis
- Cost optimization modeling
- Predictive analytics

In [ ]:
# Generate insights automatically
record_count = f"{len(df):,}"
completeness = f"{((1 - df.isnull().sum().sum() / df.size) * 100):.1f}"

# Most used mode
most_used_mode = df['transport_mode'].value_counts().index[0]

# Most expensive mode
most_expensive_mode = df.groupby('transport_mode')['cost'].mean().idxmax()

# Fastest mode
if 'transit_days' in df.columns:
    fastest_mode = df.groupby('transport_mode')['transit_days'].mean().idxmin()
else:
    fastest_mode = "N/A"

print("✅ ANALYSIS COMPLETE")
print("\n📊 Quick Stats:")
print(f"  • Total Records: {record_count}")
print(f"  • Data Completeness: {completeness}%")
print(f"  • Most Used Mode: {most_used_mode}")
print(f"  • Most Expensive Mode: {most_expensive_mode}")
print(f"  • Fastest Mode: {fastest_mode}")